# 서울시 열린데이터 년도별 따릉이 대여건수 합계 계산

## 스파크 기본설정

In [1]:
import os
import sys

#home=os.path.expanduser("~") # HOME이 설정되어 있지 않으면 expanduser('~')를 사용한다.
#osn.environ["PYSPARK_PYTHON"] = "/usr/bin/python"
os.environ["SPARK_HOME"]=os.path.join(os.path.expanduser("~"),r"C:\Users\user\spark-2.0.0-bin-hadoop2.7\spark-2.0.0-bin-hadoop2.7")
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

## csv파일 읽어와서 dataframe에 schema 만들어서 적용

In [15]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myBikeSchema = StructType([
    StructField("RENT_DATE", StringType(), True),
    StructField("RENT_COUNT", StringType(), True)
])

df = spark.read.format("csv").schema(myBikeSchema)\
    .load("seoul_bike_rent_count(2018_2019.03).csv")

AnalysisException: u'Path does not exist: file:/C:/Users/user/MyStudySpace/2019-2/\ube45\ub370\uc774\ud130\ubd84\uc11d/assignment/seoul_bike_rent_count(2018_2019.03).csv;'